In [22]:
!pip install numpy
!wget https://raw.githubusercontent.com/Mehrdadghassabi/Gracc/master/BCSThesis/Source/circuits/circuit1.txt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
--2022-07-12 23:48:21--  https://raw.githubusercontent.com/Mehrdadghassabi/Gracc/master/BCSThesis/Source/circuits/circuit1.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.109.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 69 [text/plain]
Saving to: ‘circuit1.txt.1’

circuit1.txt.1      100%[===================>]      69  --.-KB/s    in 0s      

2022-07-12 23:48:22 (2.91 MB/s) - ‘circuit1.txt.1’ saved [69/69]



In [23]:
from scipy.sparse import csr_matrix
from scipy.sparse.csgraph import minimum_spanning_tree
import numpy as np

In [24]:
# this function get the example circuit information
# from .txt file by parsing it
# 
# returns an array which contains
#    nodesnumber ,
#    edgesnumber ,
#    an array which contains resistors of each edge ,
#    an array which contains batterys vlotage of each edge ,
#    an array which contains capacitors of each edge ,
#    an array which contains selves of each edge &
#    Kirchoff graph adjacency matrix
def parsing_cicuit(circ):
     nodesnumber = 0
     edgesnumber = 0
     resistors = np.zeros((6), dtype=int)
     batterys = np.zeros((6), dtype=int)
     capacitors = np.zeros((6), dtype=int)
     selves = np.zeros((6), dtype=int)
     i = 0
     # i is the loop variable
     with open(circ) as f:
         for line in f:
            # for each line in the .txt file get the line number i
            omitnewlines = line.replace('\n', '')
            # remove newlines
            arrstr = omitnewlines.split(" ")
            arrint = [int(numeric_string) for numeric_string in arrstr]
            # in each lines there are some numbers
            # convert the line which is a string to array of numbers
            if i == 0 :
              # if its the first line it determines 
              # number of nodes & edges
              nodesnumber = arrint[0]
              edgesnumber = arrint[1]
              adjacencymatrix = np.zeros([nodesnumber,nodesnumber], dtype=int)
            else :
              # else it means that this line is an information about an edge
              # the origin and the destanation node
              # & information that an edge contains
              xmat = arrint[0]
              ymat = arrint[1]
              adjacencymatrix[xmat][ymat] = 1
              resistors[i-1] = arrint[2]
              batterys[i-1] = arrint[3]
              capacitors[i-1] = arrint[4]
              selves[i-1] = arrint[5]
            i = i + 1
     # return them as an array ! !!
     circ = [nodesnumber,edgesnumber,resistors,batterys,
             capacitors,selves,adjacencymatrix]

     return circ

In [25]:
circuit1 = parsing_cicuit('circuit1.txt')
print(circuit1[6])


[[0 1 1 1]
 [0 0 1 1]
 [0 0 0 0]
 [0 0 0 0]]
